# Phase 3.4: Complete Model Registry Workflow

This comprehensive notebook demonstrates a **real-world ML workflow**:
1. Train initial model
2. Register in registry
3. Promote to Staging
4. Test in Staging
5. Promote to Production
6. Train improved model
7. Compare and promote new version

## The MLOps Workflow

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   DEVELOP   │────>│   STAGING   │────>│ PRODUCTION  │
│             │     │             │     │             │
│ Train Model │     │ Run Tests   │     │ Serve Users │
│ Log to MLflow     │ Validate    │     │ Monitor     │
│ Register    │     │ Compare     │     │             │
└─────────────┘     └─────────────┘     └─────────────┘
```

## Learning Goals
- Understand the full ML lifecycle
- Practice promoting models through stages
- Compare model versions programmatically
- Implement production deployment patterns

## Step 1: Import Libraries and Setup

In [ ]:
# MLflow imports
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# sklearn imports
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Data handling and utilities
import pandas as pd
import time
import os

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")
print("Ready to learn the complete registry workflow!")

## Step 2: Connect to MLflow

In [ ]:
# Get MLflow tracking server URL
TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI", "http://localhost:5000")

# Connect to MLflow
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment("phase3-workflow")

# Model name for this workflow
MODEL_NAME = "iris-production-workflow"

# Create client
client = MlflowClient()

print(f"Connected to MLflow at: {TRACKING_URI}")
print(f"Experiment: phase3-workflow")
print(f"Model name: {MODEL_NAME}")

## Step 3: Define Helper Functions

We'll create reusable functions for common operations. This is how you'd organize code in a real project.

In [ ]:
def train_model(params, run_name):
    """
    Train and log a model to MLflow.
    
    Args:
        params: Dictionary of model parameters
        run_name: Name for the MLflow run
    
    Returns:
        tuple: (run_id, accuracy)
    """
    # Load data
    iris = load_iris()
    X = pd.DataFrame(iris.data, columns=iris.feature_names)
    y = iris.target
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Train and log
    with mlflow.start_run(run_name=run_name) as run:
        model = RandomForestClassifier(**params, random_state=42)
        model.fit(X_train, y_train)
        
        accuracy = accuracy_score(y_test, model.predict(X_test))
        mlflow.log_params(params)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.sklearn.log_model(model, "model")
        
        return run.info.run_id, accuracy


def register_model(run_id, model_name):
    """
    Register a model from a run.
    
    Args:
        run_id: The MLflow run ID
        model_name: Name for the registered model
    
    Returns:
        str: The version number
    """
    model_uri = f"runs:/{run_id}/model"
    result = mlflow.register_model(model_uri, model_name)
    return result.version


def promote_to_stage(model_name, version, stage, archive_existing=False):
    """
    Promote a model version to a stage.
    
    Args:
        model_name: Name of the registered model
        version: Version number to promote
        stage: Target stage (Staging, Production, Archived)
        archive_existing: Whether to archive existing versions in the target stage
    """
    client.transition_model_version_stage(
        name=model_name,
        version=str(version),
        stage=stage,
        archive_existing_versions=archive_existing
    )


def load_and_test(model_name, stage):
    """
    Load a model from a stage and run tests.
    
    Args:
        model_name: Name of the registered model
        stage: Stage to load from
    
    Returns:
        float: Test accuracy
    """
    model = mlflow.pyfunc.load_model(f"models:/{model_name}/{stage}")
    
    # Load test data
    iris = load_iris()
    X_test = pd.DataFrame(iris.data[:10], columns=iris.feature_names)
    y_test = iris.target[:10]
    
    predictions = model.predict(X_test)
    accuracy = (predictions == y_test).mean()
    
    return accuracy


print("Helper functions defined:")
print("  - train_model(params, run_name)")
print("  - register_model(run_id, model_name)")
print("  - promote_to_stage(model_name, version, stage)")
print("  - load_and_test(model_name, stage)")

## Step 4: Clean Up (For Demo)

In [ ]:
# Clean up existing model for a fresh demo
try:
    client.delete_registered_model(MODEL_NAME)
    print(f"Cleaned up existing model: {MODEL_NAME}")
except:
    print(f"No existing model to clean up.")

## Workflow Step 1: Train Initial Model

Start with a baseline model using conservative hyperparameters.

In [ ]:
print("="*70)
print("ML MODEL REGISTRY WORKFLOW")
print("="*70)

print("\n[Step 1] Training Initial Model")
print("-" * 50)

# Conservative initial parameters
params_v1 = {"n_estimators": 50, "max_depth": 5}

run_id_v1, accuracy_v1 = train_model(params_v1, "initial-model")

print(f"  Parameters: {params_v1}")
print(f"  Accuracy: {accuracy_v1:.4f}")
print(f"  Run ID: {run_id_v1[:8]}...")

## Workflow Step 2: Register Model

Add the model to the Model Registry for versioning and lifecycle management.

In [ ]:
print("\n[Step 2] Registering Model in Registry")
print("-" * 50)

version_v1 = register_model(run_id_v1, MODEL_NAME)

print(f"  Registered as: {MODEL_NAME}")
print(f"  Version: {version_v1}")

# Wait for registration to complete
time.sleep(1)

## Workflow Step 3: Promote to Staging

Move the model to Staging for testing before production.

In [ ]:
print("\n[Step 3] Promoting to Staging for Testing")
print("-" * 50)

promote_to_stage(MODEL_NAME, version_v1, "Staging")

print(f"  Version {version_v1} -> Staging")
print("  (Model is now available for validation testing)")

## Workflow Step 4: Test in Staging

Run validation tests on the staging model before promoting to production.

In [ ]:
print("\n[Step 4] Running Tests in Staging")
print("-" * 50)

staging_accuracy = load_and_test(MODEL_NAME, "Staging")

print(f"  Staging test accuracy: {staging_accuracy:.4f}")

# Define acceptance criteria
ACCEPTANCE_THRESHOLD = 0.8

if staging_accuracy >= ACCEPTANCE_THRESHOLD:
    print(f"  Tests PASSED! (threshold: {ACCEPTANCE_THRESHOLD})")
    staging_passed = True
else:
    print(f"  Tests FAILED! (threshold: {ACCEPTANCE_THRESHOLD})")
    staging_passed = False

## Workflow Step 5: Promote to Production

If tests pass, promote the model to Production.

In [ ]:
print("\n[Step 5] Promoting to Production")
print("-" * 50)

if staging_passed:
    promote_to_stage(MODEL_NAME, version_v1, "Production")
    print(f"  Version {version_v1} -> Production")
    print("  (Model is now serving live traffic)")
else:
    print("  Skipping - staging tests did not pass")

## Workflow Step 6: Train Improved Model

Train a new model with better hyperparameters.

In [ ]:
print("\n[Step 6] Training Improved Model")
print("-" * 50)

# More aggressive parameters for improved model
params_v2 = {"n_estimators": 100, "max_depth": 10}

run_id_v2, accuracy_v2 = train_model(params_v2, "improved-model")

print(f"  Parameters: {params_v2}")
print(f"  Accuracy: {accuracy_v2:.4f}")
print(f"  Run ID: {run_id_v2[:8]}...")

## Workflow Step 7: Register New Version

In [ ]:
print("\n[Step 7] Registering Improved Model")
print("-" * 50)

version_v2 = register_model(run_id_v2, MODEL_NAME)

print(f"  Registered as: {MODEL_NAME}")
print(f"  Version: {version_v2}")

time.sleep(1)

## Workflow Step 8: Compare Models

Compare the new model against the current production model.

In [ ]:
print("\n[Step 8] Comparing Models")
print("-" * 50)

print(f"  Production (v{version_v1}): accuracy = {accuracy_v1:.4f}")
print(f"  Candidate  (v{version_v2}): accuracy = {accuracy_v2:.4f}")

improvement = accuracy_v2 - accuracy_v1
print(f"\n  Improvement: {improvement:+.4f}")

if improvement > 0:
    print("  Result: New model is BETTER")
elif improvement < 0:
    print("  Result: New model is WORSE")
else:
    print("  Result: Models are EQUAL")

## Workflow Step 9: Make Promotion Decision

Decide whether to promote the new model to production.

In [ ]:
print("\n[Step 9] Making Promotion Decision")
print("-" * 50)

if accuracy_v2 > accuracy_v1:
    print("  Decision: PROMOTE new model to Production")
    
    # First, send to Staging for testing
    promote_to_stage(MODEL_NAME, version_v2, "Staging")
    print(f"\n  Version {version_v2} -> Staging")
    
    # Run staging tests
    staging_accuracy = load_and_test(MODEL_NAME, "Staging")
    print(f"  Staging tests passed (accuracy: {staging_accuracy:.4f})")
    
    # Promote to Production (archive_existing=True auto-archives old production)
    promote_to_stage(MODEL_NAME, version_v2, "Production", archive_existing=True)
    print(f"\n  Version {version_v2} -> Production")
    print(f"  Version {version_v1} -> Archived (automatically)")
    
else:
    print("  Decision: KEEP current Production model")
    
    # Archive the candidate since it's not better
    promote_to_stage(MODEL_NAME, version_v2, "Archived")
    print(f"\n  Version {version_v2} -> Archived (not better than production)")

## Final Registry State

In [ ]:
print("\n" + "="*70)
print("FINAL REGISTRY STATE")
print("="*70)

print(f"\nModel: {MODEL_NAME}")
print("-" * 50)

for mv in client.search_model_versions(f"name='{MODEL_NAME}'"):
    # Get run metrics
    run = client.get_run(mv.run_id)
    accuracy = run.data.metrics.get("accuracy", 0)
    
    print(f"  Version {mv.version}: {mv.current_stage:12s} (accuracy: {accuracy:.4f})")

## Load and Verify Production Model

In [ ]:
print("\n" + "-" * 50)
print("Verifying Production Model")
print("-" * 50)

# Load the current production model
prod_model = mlflow.pyfunc.load_model(f"models:/{MODEL_NAME}/Production")

print("\nProduction model loaded successfully!")
print("Ready for inference.")

# Quick test
iris = load_iris()
test_sample = pd.DataFrame([iris.data[0]], columns=iris.feature_names)
prediction = prod_model.predict(test_sample)

print(f"\nSample prediction: {iris.target_names[prediction[0]]}")

## Summary: Complete MLOps Workflow

### The 9-Step Workflow

```
1. Train Model       -> Create initial model
2. Register          -> Add to Model Registry
3. Stage             -> Move to Staging
4. Test              -> Run validation tests
5. Deploy            -> Promote to Production
6. Iterate           -> Train improved model
7. Register          -> Create new version
8. Compare           -> Evaluate against production
9. Promote/Archive   -> Deploy or discard
```

### Key Patterns

| Pattern | Purpose |
|---------|----------|
| Staging tests | Validate before production |
| Model comparison | Data-driven promotion decisions |
| Auto-archive | Clean up old production versions |
| Stage-based loading | Consistent production access |

### Production Code Pattern

```python
# Your inference service just needs:
model = mlflow.pyfunc.load_model("models:/my-model/Production")
predictions = model.predict(data)

# When you promote a new model, this code automatically uses it!
```

In [ ]:
print("="*70)
print("Registry Workflow Tutorial Complete!")
print("="*70)
print(f"\nView at: {TRACKING_URI}/#/models/{MODEL_NAME}")
print("\nWhat you learned:")
print("  1. The complete ML model lifecycle")
print("  2. How to structure helper functions for MLOps")
print("  3. How to test models in Staging before Production")
print("  4. How to compare model versions programmatically")
print("  5. How to safely promote new models to Production")
print("  6. How to archive old models automatically")